In [1]:
import pandas as pd
import json

for dim in [5, 30]:
    df = pd.read_csv(f"./data/landscape_data/ELA_{dim}D.csv", index_col=0)
    # problem_instance_id, problem_instance_label, group, feat
    problem_instance_dict = {}

    def get_problem_class_group(f):
        if f >=1 and f<=5:
            return 0 #'separable'
        if f >=6 and f<=9:
            return 1 #'low or moderate conditioning'
        if f >=10 and f<=14:
            return 2 #'high conditioning and unimodal'
        if f >=15 and f<=19:
            return 3 #'multi modal with adequate global structure'
        if f >=19 and f<=24:
            return 4 #'multi modal with whit global structure'

    i = 0
    data = []
    for iid in range(1,6):
        for fid in range(1, 25):
            problem_instance_dict[str(fid)+"_"+str(iid)] = i
            feat = list(df.loc[str(fid)+"_"+str(iid)])      
            data.append([i, str(fid)+"_"+str(iid), get_problem_class_group(fid), feat])
            i+=1

    df_new = pd.DataFrame(data=data, columns=['node_id', 'label', 'group', 'ela'])
    df_new.to_csv(f"./data/graph_data/problem/problem_instance_{dim}D.csv", index=False)


    json_str = json.dumps(problem_instance_dict, indent=4)
    with open(f'./data/graph_data/problem/problem_instances_dict_{dim}D.json', 'w') as f:
        f.write(json_str)

In [2]:
import pandas as pd
import json
import numpy as np

def get_encoding(dict, algo):
    with open(f'./data/graph_data/algo/{algo}-parameter-encoding.json', 'r') as json_file:
        algo_dict = json.load(json_file,)
    arr = []
    for item in dict.items():
        f = algo_dict[item[0]][str(item[1])]
        arr.append(f)
    return(arr)


for algo in ['modCMA', 'modDE']:
    print(algo)
    df = pd.read_csv(f"./data/configuration_info/{algo}_conf_grid.csv", index_col=0)
    df = df.replace({np.nan: 'N'})
    data = []
    n_algo = 324 if algo == 'modCMA' else 576
    for i in range(0, n_algo):
        row = df.iloc[i]
        feat = get_encoding(row, algo)
        data.append([i, f'{algo}_conf-{i}', feat])

    df_new = pd.DataFrame(data=data, columns=['node_id', 'label', 'feat'])
    df_new.to_csv(f"./data/graph_data/algo/algo_instance_{algo}.csv", index=False)


modCMA
modDE


In [3]:
import pandas as pd
import json
import numpy as np

for algo in ['modCMA-ES', 'modDE']:
        for dim in [5, 30]:
               for budget in [50*dim, 100*dim, 300*dim, 500*dim, 1000*dim, 1500*dim]: 
                        with open(f'./data/graph_data/problem/problem_instances_dict_{dim}D.json', 'r') as json_file:
                                problem_dict = json.load(json_file)

                                data = []
                                n_algo = 324 if algo == 'modCMA-ES' else 576
                                for conf_id in range(0, n_algo):
                                        df = pd.read_csv(f"./data/performance_data/{algo}/unlog/budget_{budget}_conf_{conf_id}_{dim}D.csv", index_col=0)
                                        for iid in range(1,6):
                                                for fid in range(1, 25):
                                                        problem = str(fid)+"_"+str(iid)
                                                        score = df.loc[problem]['target']

                                                        score_log = -8 if score <= 10**(-8) else np.log10(score)
                                                        problem_id = problem_dict[problem]
                                                        data.append([conf_id, problem_id, score_log])

                                df_new = pd.DataFrame(data=data, columns=['src_id', 'dst_id', 'score'])
                                df_new.to_csv(f"./data/graph_data/performance/performance_edge_{algo.split('-')[0]}_{dim}_{budget}.csv", index=False)

In [9]:
import pandas as pd
import json
import numpy as np

for algo in ['modCMA-ES', 'modDE']:
        for dim in [5, 30]:
               for budget in [50*dim, 100*dim, 300*dim, 500*dim, 1000*dim, 1500*dim]: 
                        with open(f'../data/graph_data/problem/problem_instances_dict_{dim}D.json', 'r') as json_file:
                                problem_dict = json.load(json_file)

                                data = []
                                n_algo = 324 if algo == 'modCMA-ES' else 576
                                for conf_id in range(0, n_algo):
                                        df = pd.read_csv(f"../data/performance_data/{algo}/unlog/budget_{budget}_conf_{conf_id}_{dim}D.csv", index_col=0)
                                        for iid in range(1,6):
                                                for fid in range(1, 25):
                                                        problem = str(fid)+"_"+str(iid)
                                                        score = df.loc[problem]['target']

                                                        score_log = -8 if score <= 10**(-8) else np.log10(score)
                                                        problem_id = problem_dict[problem]
                                                        data.append([conf_id, problem, score_log])

                                df_new = pd.DataFrame(data=data, columns=['src_id', 'dst_id', 'score'])
                                df_new.to_csv(f"../data/graph_data/performance/err_performance_edge_{algo.split('-')[0]}_{dim}_{budget}.csv", index=False)

Generate log performance

In [1]:
import pandas as pd
import json
import numpy as np

for algo in ['modCMA-ES', 'modDE']:
    for dim in [5, 30]:
        for budget in [50*dim, 100*dim, 300*dim, 500*dim, 1000*dim, 1500*dim]: 
            # Load the problem dictionary
            with open(f'../data/graph_data/problem/problem_instances_dict_{dim}D.json', 'r') as json_file:
                problem_dict = json.load(json_file)

            # Initialize a dictionary to store scores for each problem instance
            data = {}
            n_algo = 324 if algo == 'modCMA-ES' else 576

            for conf_id in range(n_algo):
                df = pd.read_csv(f"../data/performance_data/{algo}/unlog/budget_{budget}_conf_{conf_id}_{dim}D.csv", index_col=0)

                # Iterate through problem instances (i.e., iid and fid combinations)
                for iid in range(1, 6):
                    for fid in range(1, 25):
                        problem = str(fid) + "_" + str(iid)
                        score = df.loc[problem]['target']

                        # Apply log transformation to the score
                        score_log = -8 if score == 0 else np.log10(score)

                        # Get the problem_id from the problem dictionary
                        problem_id = problem_dict[problem]

                        # Initialize the dictionary key if not present
                        if problem_id not in data:
                            data[problem_id] = {'dst_id': problem}

                        # Assign the score for this configuration as a new column
                        data[problem_id][f'conf_{conf_id}'] = score_log

            # Convert the dictionary to a DataFrame
            df_new = pd.DataFrame.from_dict(data, orient='index')

            # Save the DataFrame to CSV
            df_new.to_csv(f"../data/performance_data/{algo}/log_corr/budget_{budget}_dim_{dim}.csv", index=False)


In [1]:
import pandas as pd
import json
import numpy as np

for algo in ['modCMA-ES', 'modDE']:
    for dim in [5, 30]:
        for budget in [50*dim, 100*dim, 300*dim, 500*dim, 1000*dim, 1500*dim]: 
            # Load the problem dictionary
            with open(f'../data/graph_data/problem/problem_instances_dict_{dim}D.json', 'r') as json_file:
                problem_dict = json.load(json_file)

            # Initialize a dictionary to store scores for each problem instance
            data = {}
            n_algo = 324 if algo == 'modCMA-ES' else 576

            for conf_id in range(n_algo):
                df = pd.read_csv(f"../data/performance_data/{algo}/unlog/budget_{budget}_conf_{conf_id}_{dim}D.csv", index_col=0)

                # Iterate through problem instances (i.e., iid and fid combinations)
                for iid in range(1, 6):
                    for fid in range(1, 25):
                        problem = str(fid) + "_" + str(iid)
                        score = df.loc[problem]['target']

                        # Apply log transformation to the score
                        score_log = -8 if score <= 10**(-8) else np.log10(score)

                        # Get the problem_id from the problem dictionary
                        problem_id = problem_dict[problem]

                        # Initialize the dictionary key if not present
                        if problem_id not in data:
                            data[problem_id] = {'dst_id': problem}

                        # Assign the score for this configuration as a new column
                        data[problem_id][f'conf_{conf_id}'] = score_log

            # Convert the dictionary to a DataFrame
            df_new = pd.DataFrame.from_dict(data, orient='index')

            # Save the DataFrame to CSV
            df_new.to_csv(f"../data/performance_data/{algo}/log_err/budget_{budget}_dim_{dim}.csv", index=False)
